In [ ]:
import yfinance as yf

# Get the historical stock prices for Apple Inc.
data = yf.download('AAPL', period='max')


In [ ]:
data.head()
data.describe()


In [ ]:
# Normalize the numerical features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_numerical = data[['Close', 'Volume']]
data_numerical_scaled = scaler.fit_transform(data_numerical)

# Encode the categorical features
data_categorical = data[['Date']]
data_categorical_encoded = pd.get_dummies(data_categorical)


In [ ]:
from sklearn.model_selection import train_test_split

X = pd.concat([data_numerical_scaled, data_categorical_encoded], axis=1)
y = data['Close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
from tensorflow import keras

# Build the LSTM model
model = keras.Sequential([
    keras.layers.LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    keras.layers.LSTM(32),
    keras.layers.Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)


In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)


In [ ]:
# Get the latest stock prices
new_data = yf.download('AAPL', period='1d')

# Prepare the new data for the model
new_data_numerical = new_data[['Close', 'Volume']]
new_data_numerical_scaled = scaler.transform(new_data_numerical)
new_data_categorical = new_data[['Date']]
new_data_categorical_encoded = pd.get_dummies(new_data_categorical)

X_new = pd.concat([new_data_numerical_scaled, new_data_categorical_encoded], axis=1)

# Make a prediction
y_pred = model.predict(X_new)
print('Predicted price:', y_pred[0])
